In [2]:
!pip install git+https://github.com/huggingface/transformers -U
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb
!pip install -q -U sentencepiece

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6m11epe0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6m11epe0
  Resolved https://github.com/huggingface/transformers to commit 9ed538f2e67ee10323d96c97284cf83d44f0c507
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.

In [3]:
!git clone https://github.com/leanprover-community/mathlib4.git || echo "already cloned"

fatal: destination path 'mathlib4' already exists and is not an empty directory.
already cloned


In [4]:
from transformers import LlamaForCausalLM, CodeLlamaTokenizer, BitsAndBytesConfig
import json
import pandas as pd
import os
from transformers import AutoTokenizer

base_model_name = "codellama/CodeLlama-7b-hf"
# You can initialize the tokenizer here. For example, for BERT:
tokenizer = CodeLlamaTokenizer.from_pretrained(base_model_name)

def get_files_in_dir(path, suffix='.txt'):
    """
    Prepares a dataset for next token prediction from files with a specific suffix in the given directory and its subdirectories.

    Parameters:
    - path (str): The directory path to look for files.
    - tokenizer: Tokenizer from the transformers library.
    - suffix (str): The file suffix to filter by. Default is '.txt'.
    - MAX_CONTENT_LENGTH (int): The maximum length for the input content.

    Returns:
    - pd.DataFrame: A DataFrame with 'input' and 'target' columns for next token prediction.
    """

    # Lists for storing input sequences and target tokens
    

    # Walking through the directory and its subdirectories
    rets = []
    for dirpath, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(suffix):
                rets.append(os.path.join(dirpath, filename))
    return rets

def prepare_file_examples(file_path: str, tokenizer: CodeLlamaTokenizer, MAX_CONTENT_LENGTH=512, MIN_CONTENT_LENGTH=128):
    outs = []
    # inputs = []
    # targets = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        # Tokenize the content using the provided tokenizer
        tokens = tokenizer.tokenize(content)
        # Start at min content length
        # for i in range(0, len(tokens) - MIN_CONTENT_LENGTH):  # -1 to leave space for the target token
            # print(i, len(tokens))
        for length in range(MIN_CONTENT_LENGTH, min(MAX_CONTENT_LENGTH, len(tokens)) + 1):
            i = 0
            # print(i, length, len(tokens))
            # print(i, length, len(tokens))
            input_sequence = tokens[i:min(i + length, len(tokens))]
            if i + length < len(tokens):
                target_token = tokens[i + length]
                outs.append({'text': tokenizer.convert_tokens_to_string(input_sequence), 'label': target_token})
                
        # TODO: clean up
        for i in range(0, max(0, len(tokens) - MAX_CONTENT_LENGTH)):
            length = MAX_CONTENT_LENGTH
            input_sequence = tokens[i:min(i + length, len(tokens))]
            if i + length < len(tokens):
                target_token = tokens[i + length]
                outs.append({'text': tokenizer.convert_tokens_to_string(input_sequence), 'label': target_token})

    return outs



def prepare_json(files):#, tokenizer, MAX_CONTENT_LENGTH):
	# r = []
    d = {"train": []}
	# with jsonlines.open('data_simple/test.jsonl', mode='w') as writer:
    for i, file in enumerate(files):
        # with open(file, 'r', encoding='utf-8', errors='ignore') as file:
    # content = file.read()
        d['train'] += prepare_file_examples(file, tokenizer, MAX_CONTENT_LENGTH, MIN_CONTENT_LENGTH=128)
        json.dump(d['train'], open('data_simple/test.json', 'w'))
        print("Finished processing file", i + 1, "of", len(files), "files")
        # r.append(prepare_file_training(file))#, tokenizer, MAX_CONTENT_LENGTH)
    # json.dump(d['train'], open('data_simple/test.json', 'w'))
#  Peep to clean https://huggingface.co/docs/datasets/index



path = '../../../lean/mathlib4/Mathlib/'
# path = './mathlib4'

MAX_CONTENT_LENGTH=2_048
lean_files = get_files_in_dir(path, ".lean")
# df = prepare_file_training(lean_files[0], tokenizer, MAX_CONTENT_LENGTH, MIN_CONTENT_LENGTH=1)
# prepare_jsonl(lean_files)
# https://medium.com/@ud.chandra/instruction-fine-tuning-llama-2-with-pefts-qlora-method-d6a801ebb19

In [5]:
!mkdir data_simple || echo "data_simple dir already exists"

mkdir: cannot create directory ‘data_simple’: File exists
data_simple dir already exists


In [6]:
import numpy as np
np.random.seed(42)

n_files = len(lean_files)
n_train_files = 10
test_file_paths = np.random.choice(lean_files, n_train_files).tolist()
prepare_json(test_file_paths)

Finished processing file 1 of 10 files
Finished processing file 2 of 10 files
Finished processing file 3 of 10 files
Finished processing file 4 of 10 files
Finished processing file 5 of 10 files
Finished processing file 6 of 10 files
Finished processing file 7 of 10 files
Finished processing file 8 of 10 files
Finished processing file 9 of 10 files
Finished processing file 10 of 10 files


# QLora this boy

In [8]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={'train': ["data_simple/test.json"]})
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 53315
    })
})

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [9]:


# When prompted, paste the HF access token you created earlier.
# from huggingface_hub import notebook_login
# notebook_login()

from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

# dataset_name = "<your_hf_dataset>"
# dataset = load_dataset(dataset_name, split="train")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    # Added by Lev, remove
    bnb_4bit_use_double_quant=True,
)

device_map = {"": 0}

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,

    # Added by Lev, remove
    load_in_4bit=True,
    # use_auth_token=True
)
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf",
#     device_map='auto',
#     # load_in_4bit=True,
#     device_map=device_map,
#     trust_remote_code=True,
#     quantization_config=bnb_config
# )
base_model = model
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

# max_seq_length = 1_024 # TODO: TUNE!
max_seq_length = 512 # TODO: TUNE!

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)
# TODO: now we are just out of memory...

2023-10-02 14:03:48.890901: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 14:03:48.932215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 14:03:49.372425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]